In [1]:
#INSTALLING THE KAGGLE API

!pip install kaggle #installing the kaggle API
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
#copying the .json file to the required directory
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json


    100% |████████████████████████████████| 61kB 2.3MB/s 
    100% |████████████████████████████████| 235kB 7.3MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/44/2c/df/22a6eeb780c36c28190faef6252b739fdc47145fd87a6642d4
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
Download 100%.


In [2]:
#downloading the files
!kaggle competitions download  -c dogs-vs-cats-redux-kernels-edition -p /content/kaggle

 98% 265M/271M [00:04<00:00, 66.3MB/s]
100% 271M/271M [00:04<00:00, 66.2MB/s]
 98% 531M/544M [00:08<00:00, 69.0MB/s]
100% 544M/544M [00:08<00:00, 64.8MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 57.4MB/s]


In [0]:
#extracting the training zip file
import zipfile

local_zip = '/content/kaggle/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/utkarsh')
zip_ref.close()


In [0]:
#importing required libraries

#image precessing
import cv2
#file handling
import os
#obvious
import numpy as np
import pandas as pd
#library to know the progress of for loops
from tqdm import tqdm

In [0]:
#the path where the required files are
base_dir = '/tmp/utkarsh'
train_dir = os.path.join(base_dir, 'train')


In [0]:
#reads all the filenames
train_fnames = os.listdir(train_dir)

In [0]:
#creating y from the file names andd one hot encoding the data
y = []
for name in train_fnames:
  if(name.split(".")[0] == "dog"):
    y.append((1,0))
  else:
    y.append((0,1))
y = np.array(y)

In [8]:
y.shape

(25000, 2)

In [0]:
#loading images and returning an array containing the features
IMG_SIZE = 96
def load_images_from_folder(folder):
    images = []
    for filename in tqdm(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = img/255
        if img is not None:
            images.append(img)
    return images

In [10]:
train1 = load_images_from_folder("/tmp/utkarsh/train")
train1 = np.array(train1)

100%|██████████| 25000/25000 [00:56<00:00, 439.56it/s]


In [0]:
#splitting the data into training and testing data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train1,y,random_state=0,test_size=0.2)

In [0]:
#freeing the memory
del train1

In [13]:
x_test.shape

(5000, 96, 96, 3)

In [14]:
#importing keras and required layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

Using TensorFlow backend.


In [0]:
#binary classification model(cats and dogs)
nClasses = 2

In [0]:
#constructing the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(96,96,3)))
model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nClasses, activation='softmax'))

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 96, 96, 32)        896       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 24, 24, 64)        0         
__________

In [26]:
model.fit(x_train,y_train,epochs =50,batch_size = 100)

Epoch 1/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.6299 - acc: 0.6306
Epoch 2/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.5069 - acc: 0.7488
Epoch 3/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.4401 - acc: 0.7935
Epoch 4/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.3811 - acc: 0.8256
Epoch 5/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.3354 - acc: 0.8517
Epoch 6/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.2972 - acc: 0.8693
Epoch 7/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.2562 - acc: 0.8911
Epoch 8/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.2235 - acc: 0.9061
Epoch 9/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.1993 - acc: 0.9169
Epoch 10/50
20000/20000 [==============================] - 35s 2ms/step - loss: 0.1678 - acc: 0.9321

In [27]:
loss, acc = model.evaluate(x_test,y_test)

5000/5000 [==============================] - 4s 744us/step


In [28]:
print(acc)

0.8776


In [29]:
loss2, acc2 = model.evaluate(x_train,y_train)

20000/20000 [==============================] - 15s 728us/step


In [30]:
print(acc2)

0.99815
